In this notebook we show PROMIS Approximation application in the CRIME dataset, where the audit regions are in total 8 generated from KMeans. 

We show through visualization the initial spatial bias and the results of the mitigation proccess by using the PROMIS approach to adjust the decision boundaries

In [ ]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join("../..")))

from methods.models.optimization_model import SpatialOptimFairnessModel
from utils.data_utils import read_scanned_regs, get_y, get_pos_info_regions
from utils.results_names_utils import combine_world_info, get_train_val_test_paths
import pandas as pd
from utils.scores import get_sbi, get_fair_stat_ratios
# from utils.stats_utils import get_signif_threshold
from utils.audit_utils import get_signif_thresh_scanned_regions
from utils.plot_utils import plot_fairness_map, plot_thresholds_adjustments
import numpy as np
from utils.geo_utils import compute_polygons, filterbbox 
from sklearn.metrics import accuracy_score
from utils.plot_utils import plot_map_with_polygons


## Read Data

In [ ]:
# read data
base_path = "../../../data/"
clf_name = "xgb"
dataset_name = "crime"
partioning_type_name = "non_overlap_k_8"
fairness_notion = "equal_opportunity"

results = {}
res_desc_label, partioning_name, prediction_name = combine_world_info(
    dataset_name, partioning_type_name, clf_name
)
_, val_path_info, test_path_info = get_train_val_test_paths(
    base_path, partioning_name, prediction_name, dataset_name
)
val_regions_df = read_scanned_regs(val_path_info["regions"])
val_pred_df = pd.read_csv(val_path_info["predictions"])
val_labels_df = pd.read_csv(val_path_info["labels"])
y_pred_val = get_y(val_pred_df, "pred")
y_pred_probs_val = get_y(val_pred_df, "prob")
y_true_val = get_y(val_labels_df, "label")
test_regions_df = read_scanned_regs(test_path_info["regions"])
test_pred_df = pd.read_csv(test_path_info["predictions"])
test_labels_df = pd.read_csv(test_path_info["labels"])
y_pred_test = get_y(test_pred_df, "pred")
y_pred_probs_test = get_y(test_pred_df, "prob")
y_true_test = get_y(test_labels_df, "label")
val_points_per_region = val_regions_df["points"].tolist()
test_points_per_region = test_regions_df["points"].tolist()

# keep instances with positive labels (for equal opportunity)
pos_y_true_indices_test, pos_points_per_region_test = get_pos_info_regions(
    y_true_test, test_points_per_region
)
y_pred_pos_test = y_pred_test[pos_y_true_indices_test]
pos_test_regions_df = test_regions_df.copy()
pos_test_regions_df['points'] = pos_points_per_region_test


accuracy_test_init = accuracy_score(y_true_test, y_pred_test)
print(f"Total regions: {len(test_regions_df)}")
print(f"Total test instances: {len(y_pred_test)}")
print(f"Test accuracy: {accuracy_test_init:.4f}")

In [ ]:
# print("="*50)
# print("🧮 Input for Fairness Audit")
# print("="*50)
# print(f"  • Y Prediction        : {y_pred_test[:5]}")
# print(f"  • Y True              : {y_true_test[:5]}")
# print(f"  • Regions[0] (test)   : {test_points_per_region[0][:5]}")

# print("\n📌 Audit Hyperparameters:")
# print(f"  • Fairness Notion           : Equal Opportunity")
# print(f"  • Significance Threshold    : 0.005")
# print(f"  • Number of Alternate Worlds: 400")

In [ ]:
# compute initial statistics

N = len(y_pred_pos_test)
P = np.sum(y_pred_pos_test)
print(f'N={N} positives')
print(f'P={P} true positives') #positives being 'serious crimes' == 1 and negative class: 'non-serious' crimes = 0 (predicted by RF classifier)
test_pred_df.head()
df_scanned_regs, signif_thresh = get_signif_thresh_scanned_regions(0.005, 400, test_points_per_region, y_pred_test, y_true=y_true_test, seed=42)  
sbi_test = np.mean(df_scanned_regs['statistic'])
test_regions_df['stat'] = df_scanned_regs['statistic']
print(f"Test SBI (Equal Opportunity): {sbi_test:.4f}")
print(f"Test Significance Threshold: {signif_thresh:.4f}")
total_signif_regions = len(df_scanned_regs[df_scanned_regs['signif']])
print(f"Total Significant Regions: {total_signif_regions}")
display(df_scanned_regs)

In [ ]:
# determine bounding box for display to avoid plotting outliers

bbox_min_lon=-118.6673
bbox_min_lat=33.707
bbox_max_lon=-118.16
bbox_max_lat=34.3374

# keep instances with positive labels (for equal opportunity)

pos_test_regions_df = test_regions_df.copy()
pos_test_regions_df['points'] = pos_points_per_region_test

pos_test_regions_df['pos_pr'] = pos_test_regions_df['points'].apply(lambda pts: sum(y_pred_pos_test[pts])/len(pts) if len(pts) > 0 else 0)
PR_test = sum(y_pred_pos_test)/len(y_pred_pos_test)

pos_test_regions_df["fair_stat_ratio"], max_stat_test = get_fair_stat_ratios(
    pos_test_regions_df["stat"].to_numpy(),
    pos_test_regions_df["pos_pr"].to_numpy(),
    PR_test,
)

# keep instances in bounding box for display

sub_df = test_pred_df[test_pred_df.index.isin(pos_y_true_indices_test)]
sub_df = sub_df.reset_index(drop=True)

sub_df = filterbbox(sub_df, bbox_min_lon, bbox_min_lat, bbox_max_lon, bbox_max_lat)
sub_test_pos_regions_df = pos_test_regions_df.copy()
set_new_pts = set(sub_df.index.tolist())
sub_test_pos_regions_df['points'] = sub_test_pos_regions_df['points'].apply(lambda pts: list(set(pts) & set_new_pts))

old_2_new_idx = {}
for i, ind in enumerate(sub_df.index):
    old_2_new_idx[ind] = i

sub_df = sub_df.reset_index(drop=True)
sub_test_pos_regions_df['points'] = sub_test_pos_regions_df['points'].apply(lambda pts: [old_2_new_idx[p] for p in pts])
sub_test_pos_regions_df = compute_polygons(sub_test_pos_regions_df, sub_df)

y_pred_test_sub = get_y(sub_df, "pred")

In [ ]:
# # show the regions and the points of the unfair by design world
# plot_map_with_polygons(
#     df=sub_df,
#     y_pred=y_pred_test_sub,
#     regs_df_list=[sub_test_pos_regions_df],
#     regs_color_list=["#0000FF"],
#     title="Regions - Points - Base Model",
# )

In [ ]:
# polygons = sub_test_pos_regions_df['polygon'].tolist()
# polygons_ = [[list(pt) for pt in poly] for poly in polygons]

In [ ]:
# import json

# # Construct indiv_info list
# indiv_info = [
#     {
#         "y_pred": int(y_pred_test[i]),
#         "y_true": int(y_true_test[i]),
#         "x": None,
#         "y": None
#     }
#     for i in range(len(y_pred_test))
# ]

# # region_indices: list of lists of individual indices
# region_indices = test_points_per_region  # already in required format

# # Full audit payload
# audit_payload = {
#     "n_worlds": 400,
#     "signif_level": 0.005,
#     "equal_opp": True,
#     "indiv_info": indiv_info,
#     "region_info": [
#         {
#             "polygon": poly,
#         }
#         for poly in polygons_
#     ],
#     "region_indices": region_indices
# }

# # Pretty-print to inspect or export to JSON file
# print(json.dumps(audit_payload, indent=2))

# # Optional: save to file
# with open("audit_input.json", "w") as f:
#     json.dump(audit_payload, f, indent=2)


In [ ]:
# shouls normalized LR
plot_fairness_map(
    regs_df_list=[sub_test_pos_regions_df],
    title="XGBoost Predictions - Normalized LR",
    score_label="fair_stat_ratio",
)

In [ ]:
# show the original thresholds and the respective normalized LR with colors
figsize = (12, 6)
plot_thresholds_adjustments(
    thresholds=[0.5]*len(test_points_per_region),  
    region_sizes=sub_test_pos_regions_df['fair_stat_ratio'].to_numpy(),
    figsize=figsize,
    display_title=True,
    title="Classification Threshold per Region of the XGBoost Model"
)

In [ ]:
# print("="*50)
# print("🧪 Input for Mitigation")
# print("="*50)
# print("🔹 Data for fitting the mitigator:")
# print(f"  • Y (val) Prediction              : {y_pred_val[:5]}")
# print(f"  • Y (val) Prediction Probabilities: {y_pred_probs_val[:5]}")
# print(f"  • Y (val) True                    : {y_true_val[:5]}")
# print(f"  • Regions[0] (val)                : {val_points_per_region[0][:5]}")

# print("\n📌 Hyperparameters:")
# print(f"  • Mitigator                      : PROMIS-A (Approximation)")
# print(f"  • Fairness Notion               : Equal Opportunity")
# print(f"  • Budget                        : 5000")
# print(f"  • Max Positive Ratio Shift      : 0.1")

# print("\n" + "="*50)
# print("🧾 Data for Prediction with the Mitigated Model")
# print("="*50)
# print(f"  • Regions[0] (test)              : {test_points_per_region[0][:5]}")
# print(f"  • Y (test) Prediction Probabilities: {y_pred_probs_test[:5]}")


In [ ]:
# def pts_per_region_to_indiv_info(pts_per_region):
#     """
#     Convert points per region to individual info format.
    
#     Args:
#         pts_per_region (list of list): List of lists where each sublist contains indices of points in a region.
        
#     Returns:
#         list: List of dictionaries with individual info.
#     """
#     indiv_to_regions = {}
#     for reg_id, pts in enumerate(pts_per_region):
#         for pt in pts:
#             if pt not in indiv_to_regions:
#                 indiv_to_regions[pt] = []
#             indiv_to_regions[pt].append(reg_id)
#     indiv_ids_sorted = sorted(indiv_to_regions.keys())
#     indiv_regions_list = [indiv_to_regions[ind] for ind in indiv_ids_sorted]
#     return indiv_regions_list

# indiv_regions_ids_list_val = pts_per_region_to_indiv_info(val_points_per_region)
# indiv_regions_ids_list_test = pts_per_region_to_indiv_info(test_points_per_region)

In [ ]:
# import json

# val_lats = val_pred_df["lat"].values.tolist() 
# val_lons = val_pred_df["lon"].values.tolist()
# test_lats = test_pred_df["lat"].values.tolist()
# test_lons = test_pred_df["lon"].values.tolist()
# # 👉 Construct the MitigationRequest payload for threshold-based mitigation
# payload = {
#     "fit_indiv_info": [
#         {
#             "y_pred": int(y_pred_val[i]),
#             "y_pred_prob": float(y_pred_probs_val[i]),
#             "lat": val_lats[i],
#             "lon": val_lons[i],
#             "y_true": int(y_true_val[i]),
#             "region_ids": indiv_regions_ids_list_val[i] 
#         }
#         for i in range(len(y_pred_val))
#     ],
#     "predict_indiv_info": [
#         {
#             "y_pred": int(y_pred_test[i]),
#             "y_pred_prob": float(y_pred_probs_test[i]),
#             "lat": test_lats[i],
#             "lon": test_lons[i],
#             "y_true": int(y_true_test[i]),
#             "region_ids": indiv_regions_ids_list_test[i]
#         }
#         for i in range(len(y_pred_probs_test))
#     ],
#     "predict_region_info": [{"polygon": poly} for poly in polygons_],
# }

# # 🔽 Save each component to its own file
# with open("seperate_threshold_input_coords/fit_indiv_info.json", "w") as f:
#     json.dump(payload["fit_indiv_info"], f, indent=2)

# with open("seperate_threshold_input_coords/predict_indiv_info.json", "w") as f:
#     json.dump(payload["predict_indiv_info"], f, indent=2)

# with open("seperate_threshold_input_coords/predict_region_info.json", "w") as f:
#     json.dump(payload["predict_region_info"], f, indent=2)

In [ ]:
# apply PROMIS Approximation mitigation method (equal opportunity)
fair_model = SpatialOptimFairnessModel("promis_app")
fair_model.fit(
    points_per_region=val_points_per_region,
    y_pred=y_pred_val,
    y_true=y_true_val,
    y_pred_probs=y_pred_probs_val,
    budget=0.1, # 5000
    max_pr_shift=0.1,
    wlimit=300,
    fair_notion=fairness_notion,
    overlap=True,
    no_of_threads=0,
    verbose=1,
)

# Apply the new thresholds and get the new predictions
test_new_preds = fair_model.predict(test_points_per_region, y_pred_probs_test, apply_fit_flips=False)

In [ ]:
# keep instances with positive labels (for equal opportunity)
# and compute new statistics
test_new_preds_pos = test_new_preds[pos_y_true_indices_test]
N_test_new = len(test_new_preds_pos)
P_test_new = np.sum(test_new_preds_pos)
print(f'N={N_test_new} points')
print(f'P={P_test_new} positives') #positives being 'serious crimes' == 1 and negative class: 'non-serious' crimes = 0 (predicted by RF classifier)

df_scanned_regs, signif_thresh_test = get_signif_thresh_scanned_regions(0.005, 400, test_points_per_region, test_new_preds, y_true_test, seed=42)  
sbi_test = np.mean(df_scanned_regs['statistic'])
pos_test_regions_df['new_stat'] = df_scanned_regs['statistic']

print(f"Test SBI (Equal Opportunity): {sbi_test:.3f}")
print(f"Test Significance Threshold: {signif_thresh_test:.5f}")

total_signif_regions = len(df_scanned_regs[df_scanned_regs['signif']])
print(f"Total Significant Regions: {total_signif_regions}")
display(df_scanned_regs)

In [ ]:
# compute normalized LR and keep instances in bounding box for display
pos_test_regions_df['new_pos_pr'] = pos_test_regions_df['points'].apply(lambda pts: sum(test_new_preds_pos[pts])/len(pts))
PR_test_new = sum(test_new_preds_pos)/len(test_new_preds_pos)

pos_test_regions_df["new_fair_stat_ratio"], _ = get_fair_stat_ratios(
    pos_test_regions_df["new_stat"].to_numpy(),
    pos_test_regions_df["new_pos_pr"].to_numpy(),
    PR_test_new,
    max_stat_test
)
sub_test_pos_regions_df['new_fair_stat_ratio'] = pos_test_regions_df["new_fair_stat_ratio"]
sub_test_pos_regions_df['new_stat'] = pos_test_regions_df["new_stat"]
sub_test_pos_regions_df['new_pos_pr'] = pos_test_regions_df["new_pos_pr"]
sub_test_pos_regions_df[["pos_pr", "new_pos_pr", "fair_stat_ratio", "new_fair_stat_ratio"]]


In [ ]:
# show the original thresholds and the respective normalized LR with colors
figsize = (12, 6)
plot_thresholds_adjustments(
    thresholds=fair_model.thresholds,  # List of threshold values
    region_sizes=sub_test_pos_regions_df['new_fair_stat_ratio'].to_numpy(),
    figsize=figsize,
    display_title=True,
    title="Classification Thresholds per Region After Mitigation"
)


In [ ]:
# show the new normalized LR after mitigation
plot_fairness_map(
    regs_df_list=[sub_test_pos_regions_df],
    title="Mitigated Predictions - Normalized LR",
    score_label="new_fair_stat_ratio",
)

In [ ]:
accuracy = accuracy_score(y_true_test, test_new_preds)
print(f"Test accuracy before mitigation: {accuracy_test_init:.3f}")
print(f"Test accuracy after mitigation: {accuracy:.3f}")